In [1]:
import json
import  string 
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf 
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt") 
nltk.download("wordnet")




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

**DATA**

In [14]:
Data = {"Intents": [

             {"tag": "dailyleave",
              "patterns": [" how many leaves can I get in a month ?"],
              "responses": [" you have 1 casual and 1 sick leave in a month "]
             },
              {"tag": "greeting",
              "patterns": [ "Hi", "Hello", "Hey"],
              "responses": ["Hi there", "Hello", "Hi :)"],
             },
              {"tag": "goodbye",
              "patterns": [ "bye", "later"],
              "responses": ["Bye", "take care"]
             },
             {"tag": "federalleaves",
              "patterns": ["how many leaves on this Eid? ", "Is office open on jummatulviddah"],
              "responses": ["as per federal govt announcement there would be 4 leaves this eid ", "this leave depend on federal govt announcement"]
             }

]}

In [15]:
lm = WordNetLemmatizer()
ourClasses= []
newWords = []
documentX = []
documentY = []
for intent in Data["Intents"]:
    for pattern in intent["patterns"]:
        ournewTkns = nltk.word_tokenize(pattern)
        newWords.extend(ournewTkns)
        documentX.append(pattern)
        documentY.append(intent["tag"])
    if intent["tag"] not in ourClasses:
       ourClasses.append(intent["tag"])

newWords = [lm.lemmatize(word.lower()) for word in newWords if word not in string.punctuation] # set words to lowercase if not in punctuation
newWords = sorted(set(newWords))
print(newWords)
ourClasses = sorted(set(ourClasses))
print(ourClasses)
  

['a', 'bye', 'can', 'eid', 'get', 'hello', 'hey', 'hi', 'how', 'i', 'in', 'is', 'jummatulviddah', 'later', 'leaf', 'many', 'month', 'office', 'on', 'open', 'this']
['dailyleave', 'federalleaves', 'goodbye', 'greeting']


In [16]:
trainingData = [] 
outEmpty = [0] * len(ourClasses)
for idx, doc in enumerate(documentX):
    bagOfwords = []
    text = lm.lemmatize(doc.lower())
    for word in newWords:
        bagOfwords.append(1) if word in text else bagOfwords.append(0)

    outputRow = list(outEmpty)
    outputRow[ourClasses.index(documentY[idx])] = 1
    trainingData.append([bagOfwords, outputRow])

random.shuffle(trainingData)
trainingData = np.array(trainingData, dtype=object)

x = np.array(list(trainingData[:, 0]))
y = np.array(list(trainingData[:, 1]))


In [17]:
iShape = (len(x[0]),)
oShape = len(y[0])
ourNewModel = Sequential()

ourNewModel.add(Dense(128, input_shape=iShape, activation="relu"))
ourNewModel.add(Dropout(0.5))
ourNewModel.add(Dense(64, activation="relu"))
ourNewModel.add(Dropout(0.3))
ourNewModel.add(Dense(oShape, activation = "softmax"))
md = tf.keras.optimizers.Adam(learning_rate=0.01)
ourNewModel.compile(loss='categorical_crossentropy',
              optimizer=md,
              metrics=["accuracy"])
print(ourNewModel.summary())
ourNewModel.fit(x, y, epochs=100, verbose=1)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 128)               2816      
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_8 (Dense)             (None, 4)                 260       
                                                                 
Total params: 11,332
Trainable params: 11,332
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
1/1 [============================

In [18]:
def ourText(text):
  newtkns = nltk.word_tokenize(text)
  newtkns = [lm.lemmatize(word) for word in newtkns]
  return newtkns

def wordBag(text, vocab):
  newtkns = ourText(text)
  bagOwords = [0] * len(vocab)
  for w in newtkns:
    for idx, word in enumerate(vocab):
      if word == w:
        bagOwords[idx] = 1
  return np.array(bagOwords)

def Pclass(text, vocab, labels):
  bagOwords = wordBag(text, vocab)
  ourResult = ourNewModel.predict(np.array([bagOwords]))[0]
  newThresh = 0.2
  yp = [[idx, res] for idx, res in enumerate(ourResult) if res > newThresh]

  yp.sort(key=lambda x: x[1], reverse=True)
  newList = []
  for r in yp:
    newList.append(labels[r[0]])
  return newList

def getRes(firstlist, fJson):
  tag = firstlist[0]
  listOfIntents = fJson["Intents"]
  for i in listOfIntents:
    if i["tag"] == tag:
      Result = random.choice(i["responses"])
      break
  return Result

In [ ]:
while True:
    newMessage = input("")
    intents = Pclass(newMessage, newWords, ourClasses)
    ourResult = getRes(intents , Data)
    print(ourResult)

1/1 [==============================] - 0s 23ms/step
Hi there
1/1 [==============================] - 0s 23ms/step
 you have 1 casual and 1 sick leave in a month 
